In [4]:
import numpy as np
import tensorflow as tf
import re
import itertools
from collections import Counter
from collections import OrderedDict
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile
import csv
import cPickle as pickle
print ("test")

test


In [2]:
#filename = maybe_download('text8.zip', 31344016)
filename = os.path.abspath("text8.zip")

# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data                
    

words = read_data(filename)
print('Data size', len(words))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 40000 #10839


def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = OrderedDict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

Data size 17005207
Most common words (+UNK) [['UNK', 523133], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
3084 originated -> 12 as
3084 originated -> 5239 anarchism
12 as -> 6 a
12 as -> 3084 originated
6 a -> 12 as
6 a -> 195 term
195 term -> 2 of
195 term -> 6 a


In [3]:
# Step 4: Build and train a skip-gram model.

batch_size = 300 #128
embedding_size = 200  #128 # Dimension of the embedding vector.
skip_window = 5       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.512
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()
  saver = tf.train.Saver({"embeddings": embeddings})

# Step 5: Begin training.
num_steps = 150001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in range(top_k):
          #print(len(reverse_dictionary))
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()
  print("final_embeddings: %s" %normalized_embeddings.eval())
  
  print("dictioanry length: %d" % len(dictionary))
  print("dictionary: %s" % dictionary)
  
  saver.save(session, "./final_embeddings/model.ckpt")
  #with open("./final_embeddings/vocab.txt", "w") as f:
  #      for i in xrange(vocabulary_size):
  #          f.write("%s %d\n" % (dictionary[i], i))
  with open("./final_embeddings/vocab.txt", "w") as f:
        pickle.dump(dictionary, f)

Initialized
Average loss at step  0 :  288.292907715
Nearest to this: bataan, dv, thousand, container, jinx, borealis, collectivization, flocked,
Nearest to often: asexually, aau, dum, jadavpur, lamiales, corner, delivers, wormwood,
Nearest to all: urgell, armaments, results, ensemble, industrialists, hurry, dollfuss, abbey,
Nearest to such: mvp, jiva, diversification, rafael, aero, standing, inch, unresolved,
Nearest to to: hussein, dealt, ventral, unsubstantiated, cartoonish, laterally, poul, estado,
Nearest to if: deseret, kde, stubbornly, scriptures, hedonistic, responded, lift, byzantium,
Nearest to used: petals, notches, deduce, airfoil, norm, dialect, motherboards, tending,
Nearest to also: entrails, usaid, transclusion, bought, ciprofloxacin, wolf, comers, flyby,
Nearest to use: commander, focused, hbc, socialite, repeater, lows, salish, reality,
Nearest to united: sewer, canned, hails, causeway, trappings, cosby, heartland, accountable,
Nearest to american: theta, vet, mica, f

Average loss at step  52000 :  5.1470737437
Average loss at step  54000 :  5.15076373243
Average loss at step  56000 :  5.11918325686
Average loss at step  58000 :  5.0875403682
Average loss at step  60000 :  5.0854558537
Nearest to this: a, the, ursus, that, it, equus, tamarin, to,
Nearest to often: asexually, tamarin, ell, dum, known, vma, aau, apatosaurus,
Nearest to all: and, the, abbey, duc, that, results, aquilegia, count,
Nearest to such: mvp, UNK, and, diversification, agouti, mesoplodon, with, magnitude,
Nearest to to: and, equus, that, ursus, tamarin, upanija, agouti, the,
Nearest to if: deseret, scriptures, that, to, lift, it, kde, a,
Nearest to used: eider, aquilegia, theatres, UNK, vma, windows, petals, callithrix,
Nearest to also: callithrix, equus, UNK, ada, and, a, remixed, tamarin,
Nearest to use: focused, commander, or, and, agouti, lows, ursus, speculated,
Nearest to united: sewer, canned, cosby, in, and, tamarin, accountable, vodka,
Nearest to american: and, ursus, 

Average loss at step  112000 :  4.97570589471
Average loss at step  114000 :  5.00702498913
Average loss at step  116000 :  4.94666810429
Average loss at step  118000 :  4.9378991555
Average loss at step  120000 :  4.91211296141
Nearest to this: it, that, which, a, rajonas, ursus, rajons, an,
Nearest to often: tamarin, dum, vma, asexually, lamiales, known, ell, callithrix,
Nearest to all: anh, rajons, and, are, the, rajonas, cyg, aquilegia,
Nearest to such: well, rajons, or, and, mvp, agouti, rajonas, are,
Nearest to to: rajonas, landsmannschaft, ursus, and, equus, upanija, that, rajons,
Nearest to if: that, deseret, scriptures, it, kde, rajonas, you, lift,
Nearest to used: vma, rajonas, this, eider, aquilegia, theatres, crb, rajons,
Nearest to also: rajons, equus, rajonas, callithrix, lep, UNK, tamarin, and,
Nearest to use: or, focused, rajonas, lows, commander, ursus, agouti, hbc,
Nearest to united: rajons, sewer, canned, cosby, tamarin, landsmannschaft, vodka, of,
Nearest to america

In [4]:
#tf.reset_default_graph()
dictionary = OrderedDict()
print("dictioanry length: %d" % len(dictionary))
print("dictionary: %s" % dictionary)

with open("./final_embeddings/vocab.txt", "r") as f:
    dictionary = pickle.load(f)
print("dictioanry length: %d" % len(dictionary))
print("dictionary: %s" % dictionary)

new_embeddings = tf.get_variable("embeddings", shape = [vocabulary_size, embedding_size])
norm = tf.sqrt(tf.reduce_sum(tf.square(new_embeddings), 1, keep_dims=True))

restore = tf.train.Saver({"embeddings": new_embeddings})

with tf.Session() as sess:
    restore.restore(sess, "./final_embeddings/model.ckpt")
    norm_embeddings = new_embeddings / norm
    print("new_embeddings: %s" %norm_embeddings.eval())


dictioanry length: 0
dictionary: OrderedDict()
dictioanry length: 40000
dictionary: OrderedDict([('UNK', 0), ('the', 1), ('of', 2), ('and', 3), ('one', 4), ('in', 5), ('a', 6), ('to', 7), ('zero', 8), ('nine', 9), ('two', 10), ('is', 11), ('as', 12), ('eight', 13), ('for', 14), ('s', 15), ('five', 16), ('three', 17), ('was', 18), ('by', 19), ('that', 20), ('four', 21), ('six', 22), ('seven', 23), ('with', 24), ('on', 25), ('are', 26), ('it', 27), ('from', 28), ('or', 29), ('his', 30), ('an', 31), ('be', 32), ('this', 33), ('which', 34), ('at', 35), ('he', 36), ('also', 37), ('not', 38), ('have', 39), ('were', 40), ('has', 41), ('but', 42), ('other', 43), ('their', 44), ('its', 45), ('first', 46), ('they', 47), ('some', 48), ('had', 49), ('all', 50), ('more', 51), ('most', 52), ('can', 53), ('been', 54), ('such', 55), ('many', 56), ('who', 57), ('new', 58), ('used', 59), ('there', 60), ('after', 61), ('when', 62), ('into', 63), ('american', 64), ('time', 65), ('these', 66), ('only', 67)

new_embeddings: [[ 0.10012652 -0.0714228  -0.02289733 ...,  0.12032923 -0.00925253
   0.10401341]
 [-0.0489016  -0.03326475  0.11969579 ...,  0.04146248 -0.08580524
  -0.0744745 ]
 [-0.05911008  0.00246984 -0.01628512 ...,  0.03341895  0.01533006
   0.1009737 ]
 ..., 
 [-0.10917597 -0.02065859  0.03950845 ..., -0.04358015 -0.05217595
  -0.07356514]
 [-0.05918077  0.04198439  0.1179584  ..., -0.07828391 -0.10345297
  -0.06059229]
 [ 0.08967406 -0.06848791  0.07969987 ..., -0.09244966  0.06966498
   0.08792024]]
